In [ ]:
from datasets import load_dataset
import pandas as pd
from dotenv import load_dotenv
import openai
import os

load_dotenv()

In [ ]:
train = pd.read_csv("../data/corpus_train.csv")

In [ ]:
few_shots = train.sample(20)

In [ ]:
few_shots["label"].value_counts("label")

In [ ]:
all_data = load_dataset("csv", data_files={'train': ["../data/corpus_train.csv"], 'validation': ["../data/corpus_valid.csv"], "test": ["../data/corpus_test.csv"]})

In [ ]:
def apply_promt(inp):
    prompt = ""

    for _,row in few_shots.iterrows():
        prompt += "prompt: " + row["sentence"] + "\nlabel: " + row["label"] + "<end>\n"

    prompt += "prompt: " + inp["sentence"] + "\nlabel:"

    return {"prompt": prompt}


prompted_validation = all_data["validation"].map(apply_promt)

In [ ]:
import time

key = os.getenv("OPENAI_API_KEY")
count = 50

answers = []

for i, prompt in enumerate(prompted_validation[:20]["prompt"]):

  response = openai.Completion.create(
    engine="text-davinci-001",
    prompt=prompt,
    temperature=0,
    max_tokens=6,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["<end>"]
  )

  print(f"{i}/{len(prompted_validation)}")

  time.sleep(0.5)

  answers.append(response.choices[0].text)


answers = [x.strip() for x in answers] 

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(prompted_validation["label"][:15], answers))